In [1]:
from core import init

init()

In [15]:
url = "https://leadiq.com/c/point-technologies-inc/5a1d98e42300005400877e7d"

# Version 2: A rewrite

import os
from scrapfly import ScrapeApiResponse, ScrapeConfig, ScrapflyClient

SCRAPFLY = ScrapflyClient(key=os.environ["SCRAPFLY_KEY"])
BASE_CONFIG = {
    # Crunchbase.com requires Anti Scraping Protection bypass feature.
    # for more: https://scrapfly.io/docs/scrape-api/anti-scraping-protection
    "asp": True,
    # "format": "markdown",
    "cache": True,
}


scrapfly_result = await SCRAPFLY.async_scrape(ScrapeConfig(url, **BASE_CONFIG))


In [16]:
from pprint import pprint

pprint(scrapfly_result.content)

('<!DOCTYPE html><html lang="en"> <head><meta charset="UTF-8"><meta '
 'name="viewport" content="width=device-width, initial-scale=1, '
 'minimum-scale=1, maximum-scale=5"><link rel="icon" type="image/png" '
 'href="https://assets-global.website-files.com/60819f9be381394642b5659e/60dccb991e7b7efb0eb153f5_leadiq-icon.png"><!-- '
 'Preload --><link rel="preload" as="image" fetchpriority="high" '
 'href="/_assets/strip-texture.webp" media="(min-width: 901px)" '
 'type="image/webp"><!-- Google Tag Manager --><!-- Fonts --><title>98point6 '
 'Technologies Inc. Company Overview, Contact Details &amp; Competitors | '
 'LeadIQ</title><link rel="stylesheet" '
 'href="/_assets/fonts/Circular/circular.css"><!--[--><meta name="description" '
 'content="Learn more about 98point6 Technologies Inc.&#39;s company details, '
 'contact information, competitors, and more. Find accurate contact data '
 'easily with LeadIQ. Book a demo today."><meta name="og:description" '
 'content="Learn more about 98poi

In [18]:
import json
import re

def find_hidden_data(result: ScrapeApiResponse) -> dict:
    """
    Extract hidden web cache (Apollo Graphql framework) from Glassdoor page HTML
    It's either in NEXT_DATA script or direct apolloState js variable
    """
    # data can be in __NEXT_DATA__ cache
    data = result.selector.css("script#__NEXT_DATA__::text").get()
    if data:
        data = json.loads(data)["props"]["pageProps"]["apolloCache"]
    else:  # or in direct apolloState cache
        data = re.findall(r'apolloState":\s*({.+})};', result.content)[0]
        data = json.loads(data)

    def _unpack_apollo_data(apollo_data):
        """
        Glassdoor uses Apollo GraphQL client and the dataset is a graph of references.
        This function unpacks the __ref references to actual values.
        """

        def resolve_refs(data, root):
            if isinstance(data, dict):
                if "__ref" in data:
                    return resolve_refs(root[data["__ref"]], root)
                else:
                    return {k: resolve_refs(v, root) for k, v in data.items()}
            elif isinstance(data, list):
                return [resolve_refs(i, root) for i in data]
            else:
                return data

        return resolve_refs(apollo_data.get("ROOT_QUERY") or apollo_data, apollo_data)

    return _unpack_apollo_data(data)

cache = find_hidden_data(scrapfly_result.content)

AttributeError: 'str' object has no attribute 'selector'

In [19]:
# maybe try newspaper4k?
import newspaper
from news.scrape import remove_img_tags

article = newspaper.article(
    url,
    language="en",
    # Remove images to prevent downloading them, which crashes
    input_html=remove_img_tags(scrapfly_result.content),
    fetch_images=False,
)
article.parse()

article

In [21]:
print(article.text)

Acquisition Strategy 98point6 Technologies has been actively acquiring strategic companies like Bright.md to enhance its virtual care capabilities, presenting potential partnership opportunities for cross-selling services to a larger combined customer base.

Funding and Expansion With recent financing of $32 million and partnerships with organizations like MultiCare Health System, 98point6 is in a strong position to expand its reach, indicating potential sales opportunities for new clients seeking innovative virtual care solutions.

Technology Integration The tech stack utilized by 98point6, including tools like HubSpot Chat and Pardot, showcases a commitment to leveraging advanced technologies, creating avenues for sales discussions around integrated solutions for healthcare organizations.

Key Personnel Impact The appointment of Younggren as president brings significant experience to the team, potentially opening doors for sales conversations with prospects looking for leadership-bac